In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import ast
import json

import warnings
warnings.filterwarnings('ignore')

In [2]:
results = pd.read_json('../scraping/data/constituency/Dum dum_2.json')
results = results.T
results

,Candidate,Party,Votes
0,SAUGATA ROY,All India Trinamool Congress,483244
1,ASIM KUMAR DASGUPTA,Communist Party of India (Marxist),328310
2,TAPAN SIKDAR,Bharatiya Janata Party,254819
3,DHANANJAY (SHAKTI) MOITRA,Indian National Congress,34116
4,ASHOK KUNDU,Party for Democratic Socialism,4823
5,NARENDRA NATH GHOSH (NAREN GHOSH),Bahujan Samaj Party,4077
6,SUKHA RANJAN BANIK,Independent,3256
7,SATYABRATA BANDYOPADHYAY,Rashtriya Janadhikar Suraksha Party,1618
8,SIKHA SEN ROY,Communist Party of India (Marxist-Leninist) Re...,1544
9,None of the Above,None of the Above,16837


In [3]:
def breakItUp(df, colName):
    v = json_normalize([ast.literal_eval(json.dumps(j)) for j in df[colName].dropna()], sep='_')
    v.index = df.index[df.pop(colName).notna()]
    df = df.join(v)
    return df

In [4]:
comparison = pd.read_json('../scraping/data/constituenciesComparison2014/19: DUM DUM\n_0.json')
comparison = comparison.T
comparison = breakItUp(comparison, 'Name')
comparison.rename(columns={'data': 'Name', 'url': 'ProfileLink'}, inplace=True)

comparison['Id'] = comparison['ProfileLink'].apply(lambda data: data.split("=")[-1])

comparison

,Age,Criminal Cases,Education Level,Gender,IPC Details*,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Name,ProfileLink,Id
1,52,No,Doctorate,M,NaN,"65,00,000~ 65 Lacs+","13,36,368~ 13 Lacs+",0,Y,Party for Democratic Socialism,0,"78,36,368~ 78 Lacs+","5,15,000~ 5 Lacs+",Ashok Kundu,http://www.myneta.info/ls2014/http://myneta.in...,9794
2,68,No,Doctorate,M,NaN,"18,04,080~ 18 Lacs+","54,12,157~ 54 Lacs+",0,Y,CPI(M),0,"72,16,237~ 72 Lacs+",0~,Asim Kumar Dasgupta,http://www.myneta.info/ls2014/http://myneta.in...,9358
3,61,Yes,Graduate,M,"voluntarily causing grievous hurt(325), Assaul...","40,00,000~ 40 Lacs+","25,06,000~ 25 Lacs+",2,Y,INC,4,"65,06,000~ 65 Lacs+","1,50,000~ 1 Lacs+",Dhananjoy Moitra,http://www.myneta.info/ls2014/http://myneta.in...,9357
4,71,No,Graduate,M,NaN,"70,00,000~ 70 Lacs+","1,12,926~ 1 Lacs+",0,Y,BSP,0,"71,12,926~ 71 Lacs+",0~,Narendra Nath Ghosh,http://www.myneta.info/ls2014/http://myneta.in...,9359
5,52,No,Post Graduate,M,NaN,"13,00,000~ 13 Lacs+","22,74,024~ 22 Lacs+",0,Y,Rashtriya Janadhikar Suraksha Party,0,"35,74,024~ 35 Lacs+","10,000~ 10 Thou+",Satyabrata Bandyopadhyay,http://www.myneta.info/ls2014/http://myneta.in...,9795
6,66,No,Post Graduate,M,NaN,"34,50,000~ 34 Lacs+","1,60,02,010~ 1 Crore+",0,Y,AITC,0,"1,94,52,010~ 1 Crore+","25,00,000~ 25 Lacs+",Saugata Roy,http://www.myneta.info/ls2014/http://myneta.in...,9356
7,48,No,Graduate,F,NaN,"5,00,000~ 5 Lacs+","2,500~ 2 Thou+",0,N,CPI(ML) Red star,0,"5,02,500~ 5 Lacs+",0~,Sikha Sen Roy,http://www.myneta.info/ls2014/http://myneta.in...,9360
8,71,No,Graduate,M,NaN,"20,00,000~ 20 Lacs+","39,08,115~ 39 Lacs+",0,Y,BJP,0,"59,08,115~ 59 Lacs+",0~,Tapan Sikdar,http://www.myneta.info/ls2014/http://myneta.in...,9361


In [5]:
tableMapper = {
    "All India Trinamool Congress": "AITC",
    "Communist Party of India  (Marxist)": "CPI(M)",
    "Bharatiya Janata Party": "BJP",
    "Indian National Congress": "INC",
    "Party for Democratic Socialism": "Party for Democratic Socialism",
    "Bahujan Samaj Party": "BSP",
    "Independent": None,
    "Rashtriya Janadhikar Suraksha Party": "Rashtriya Janadhikar Suraksha Party",
    "Communist Party of India (Marxist-Leninist) Red Star": "CPI(ML) Red star",
    "None of the Above": None
}

In [6]:
for index, row in results.iterrows():
    foo = comparison.loc[comparison['Party Code'] == tableMapper[row['Party']]]
    
    if len(foo) == 0:
        foo = pd.DataFrame([["NaN"] * len(comparison.columns)], columns=comparison.columns)
        
    foo['Votes'] = row['Votes']
    foo['Name'] = row['Candidate']
    foo['Party Code'] = row['Party']

    try:
        completeTable = completeTable.append(foo)
    except:
        completeTable = foo

completeTable = completeTable.head(4)
        
completeTable['Immovable Assets'] = completeTable['Immovable Assets'].apply(lambda data: data.split("~")[0].replace(',', ''))
completeTable['Movable Assets'] = completeTable['Movable Assets'].apply(lambda data: data.split("~")[0].replace(',', ''))
completeTable['Total Assets'] = completeTable['Total Assets'].apply(lambda data: data.split("~")[0].replace(',', ''))
completeTable['Total Liabilities'] = completeTable['Total Liabilities'].apply(lambda data: data.split("~")[0].replace(',', ''))

completeTable = completeTable.apply(pd.to_numeric, errors='ignore')

totalVoters = completeTable['Votes'].sum()
print(totalVoters)

completeTable

1100489


,Age,Criminal Cases,Education Level,Gender,IPC Details*,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Name,ProfileLink,Id,Votes
6,66,No,Post Graduate,M,NaN,3450000,16002010,0,Y,All India Trinamool Congress,0,19452010,2500000,SAUGATA ROY,http://www.myneta.info/ls2014/http://myneta.in...,9356,483244
2,68,No,Doctorate,M,NaN,1804080,5412157,0,Y,Communist Party of India (Marxist),0,7216237,0,ASIM KUMAR DASGUPTA,http://www.myneta.info/ls2014/http://myneta.in...,9358,328310
8,71,No,Graduate,M,NaN,2000000,3908115,0,Y,Bharatiya Janata Party,0,5908115,0,TAPAN SIKDAR,http://www.myneta.info/ls2014/http://myneta.in...,9361,254819
3,61,Yes,Graduate,M,"voluntarily causing grievous hurt(325), Assaul...",4000000,2506000,2,Y,Indian National Congress,4,6506000,150000,DHANANJAY (SHAKTI) MOITRA,http://www.myneta.info/ls2014/http://myneta.in...,9357,34116


In [7]:
educationMapper = {
    "Post Doctorate": 1,
    "Doctorate": 2,
    "Post Graduate": 3,
    "Graduate": 4
}

In [8]:
payload = completeTable.head(4)

del payload['IPC Details*']
del payload['Criminal Cases']
del payload['ProfileLink']
del payload['Id']

payload['PAN Given(Y or N)'] = payload['PAN Given(Y or N)'].apply(lambda data: 1 if 'Y' else 0)
payload['Education Level'] = payload['Education Level'].apply(lambda data: educationMapper[data])
payload['Votes'] = payload['Votes'].apply(lambda data: data/totalVoters)

payload

,Age,Education Level,Gender,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Name,Votes
6,66,3,M,3450000,16002010,0,1,All India Trinamool Congress,0,19452010,2500000,SAUGATA ROY,0.439118
2,68,2,M,1804080,5412157,0,1,Communist Party of India (Marxist),0,7216237,0,ASIM KUMAR DASGUPTA,0.298331
8,71,4,M,2000000,3908115,0,1,Bharatiya Janata Party,0,5908115,0,TAPAN SIKDAR,0.231551
3,61,4,M,4000000,2506000,2,1,Indian National Congress,4,6506000,150000,DHANANJAY (SHAKTI) MOITRA,0.031001


In [9]:
mplads = pd.read_csv('../scraping/data/mplads.csv')
mplads = mplads.iloc[:, : 9]

mplads = mplads.apply(pd.to_numeric, errors='ignore')
mplads["Constituency"] = mplads["MPName"].apply(lambda data: data.split(" ")[-1])
mplads["MPName"] = mplads["MPName"].apply(lambda data: ' '.join(data.split(" ")[:-1]))
mplads["Constituency"] = mplads["Constituency"].apply(lambda data: data.replace("-", " "))

mplads.head()

,MPName,Entitlement,FundReceivedGOI,AmountAvailable,WorksRecommCost,WSCost,ActualExpenditureIncurred,UtilizationOverRelease,UnspentBalance,Constituency
0,Shri Idris Ali,25.0,25.0,25.76,28.03,24.74,24.51,96.06,1.24,BASIRHAT
1,Km. Arpita Ghosh,25.0,25.0,27.94,29.25,26.81,25.04,98.15,2.90,BALURGHAT
2,Dr.(Smt.) Mamtaz Sanghamita,25.0,20.0,20.17,20.51,19.85,16.47,80.37,3.69,BARDHAMAN DURGAPUR
3,Shri Sisir Kumar Adhikari,25.0,25.0,25.16,24.87,23.99,21.81,85.25,3.35,KANTHI
4,Prof. Saugata Roy,25.0,25.0,25.67,26.58,24.84,23.88,93.54,1.79,DUM DUM


In [10]:
partyCodeMapper = {
    "All India Trinamool Congress": 100,
    "Communist Party of India  (Marxist)": 101,
    "Bharatiya Janata Party": 102,
    "Indian National Congress": 104
}

payload["Gender"] = payload["Gender"].apply(lambda data: 1 if data == "M" else 2)
payload["Party Code"] = payload["Party Code"].apply(lambda data: partyCodeMapper[data])
del payload["Name"]

winnerVotePercentage = payload["Votes"].max()
payload["Winner"] = payload["Votes"].apply(lambda data: 1 if data == winnerVotePercentage else 0)

x_train = payload
y_train = payload[["Winner"]].copy()
del x_train["Winner"]

x_train

,Age,Education Level,Gender,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Votes
6,66,3,1,3450000,16002010,0,1,100,0,19452010,2500000,0.439118
2,68,2,1,1804080,5412157,0,1,101,0,7216237,0,0.298331
8,71,4,1,2000000,3908115,0,1,102,0,5908115,0,0.231551
3,61,4,1,4000000,2506000,2,1,104,4,6506000,150000,0.031001


In [11]:
y_train

,Winner
6,1
2,0
8,0
3,0


In [12]:
x_train.to_csv('x_train.csv', sep=',')
y_train.to_csv('y_train.csv', sep=',')